# 03_modeling.ipynb - DAY 14: Model Comparison

**Objective**: Compare all trained models across multiple metrics:
- ROC-AUC
- PR-AUC  
- Top-k anomalies
- False positives

**Models Evaluated**:
1. Isolation Forest (Tuned)
2. LSTM Autoencoder
3. Deep SVDD

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc, precision_recall_curve, confusion_matrix
from pathlib import Path
import json
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📊 DAY 14: COMPREHENSIVE MODEL COMPARISON")
print("=" * 50)

## 1. Load All Model Results

In [ ]:
def load_all_results():
    """Load results from all three models"""
    base_path = Path("../")
    
    results = {}
    
    # 1. Isolation Forest Results
    try:
        if_data = pd.read_csv(base_path / "data/processed/isolation_forest_tuned_results.csv")
        results['isolation_forest'] = {
            'scores': if_data['anomaly_score'].values,
            'labels': if_data['is_anomaly'].values,
            'name': 'Isolation Forest'
        }
        print("✅ Loaded Isolation Forest results")
    except Exception as e:
        print(f"❌ Failed to load Isolation Forest: {e}")
    
    # 2. LSTM Autoencoder Results
    try:
        lstm_dir = base_path / "data/processed/lstm"
        recon_train = np.load(lstm_dir / "recon_train.npy")
        recon_val = np.load(lstm_dir / "recon_val.npy")
        ae_scores = np.concatenate([recon_train, recon_val])
        
        # Load threshold
        with open(base_path / "models/threshold_config.json", "r") as f:
            config = json.load(f)
        threshold = config['selected_threshold']
        ae_labels = (ae_scores > threshold).astype(int)
        
        results['autoencoder'] = {
            'scores': ae_scores,
            'labels': ae_labels,
            'threshold': threshold,
            'name': 'LSTM Autoencoder'
        }
        print("✅ Loaded Autoencoder results")
    except Exception as e:
        print(f"❌ Failed to load Autoencoder: {e}")
    
    # 3. Deep SVDD Results
    try:
        svdd_scores = np.load(lstm_dir / "svdd_scores.npy")
        svdd_labels = np.load(lstm_dir / "svdd_labels.npy")
        
        results['svdd'] = {
            'scores': svdd_scores,
            'labels': svdd_labels,
            'name': 'Deep SVDD'
        }
        print("✅ Loaded Deep SVDD results")
    except Exception as e:
        print(f"❌ Failed to load Deep SVDD: {e}")
    
    return results

# Load all results
model_results = load_all_results()
print(f"\n📈 Loaded {len(model_results)} models for comparison")

## 2. ROC-AUC Comparison

In [ ]:
def compute_roc_metrics(results):
    """Compute ROC curves and AUC for all models"""
    roc_data = {}
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
    
    for i, (model_name, data) in enumerate(results.items()):
        scores = data['scores']
        labels = data['labels']
        
        # Convert scores to anomaly scores (higher = more anomalous)
        if model_name == 'svdd':
            anomaly_scores = -scores  # SVDD: lower = more anomalous
        else:
            anomaly_scores = scores
        
        # ROC Curve
        fpr, tpr, _ = roc_curve(labels, anomaly_scores)
        roc_auc = auc(fpr, tpr)
        
        roc_data[model_name] = {
            'fpr': fpr,
            'tpr': tpr,
            'auc': roc_auc
        }
        
        # Plot ROC curve
        ax1.plot(fpr, tpr, color=colors[i], lw=2,
                label=f'{data["name"]} (AUC = {roc_auc:.3f})')
    
    # ROC plot formatting
    ax1.plot([0, 1], [0, 1], 'k--', lw=1)
    ax1.set_xlim([0.0, 1.0])
    ax1.set_ylim([0.0, 1.05])
    ax1.set_xlabel('False Positive Rate')
    ax1.set_ylabel('True Positive Rate')
    ax1.set_title('ROC Curves Comparison')
    ax1.legend(loc="lower right")
    ax1.grid(True, alpha=0.3)
    
    # AUC Bar Chart
    model_names = [results[k]['name'] for k in roc_data.keys()]
    auc_scores = [roc_data[k]['auc'] for k in roc_data.keys()]
    
    bars = ax2.bar(model_names, auc_scores, color=colors[:len(model_names)], alpha=0.7)
    ax2.set_ylabel('ROC-AUC Score')
    ax2.set_title('ROC-AUC Comparison')
    ax2.set_ylim(0, 1)
    
    # Add value labels on bars
    for bar, score in zip(bars, auc_scores):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{score:.3f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('../models/trained_models/roc_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    return roc_data

roc_results = compute_roc_metrics(model_results)

## 3. Precision-Recall AUC Comparison

In [ ]:
def compute_pr_metrics(results):
    """Compute Precision-Recall curves and AUC"""
    pr_data = {}
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
    
    for i, (model_name, data) in enumerate(results.items()):
        scores = data['scores']
        labels = data['labels']
        
        # Convert scores to anomaly scores
        if model_name == 'svdd':
            anomaly_scores = -scores
        else:
            anomaly_scores = scores
        
        # PR Curve
        precision, recall, _ = precision_recall_curve(labels, anomaly_scores)
        pr_auc = auc(recall, precision)
        
        pr_data[model_name] = {
            'precision': precision,
            'recall': recall,
            'auc': pr_auc
        }
        
        # Plot PR curve
        ax1.plot(recall, precision, color=colors[i], lw=2,
                label=f'{data["name"]} (AUC = {pr_auc:.3f})')
    
    # PR plot formatting
    ax1.set_xlim([0.0, 1.0])
    ax1.set_ylim([0.0, 1.05])
    ax1.set_xlabel('Recall')
    ax1.set_ylabel('Precision')
    ax1.set_title('Precision-Recall Curves')
    ax1.legend(loc="lower left")
    ax1.grid(True, alpha=0.3)
    
    # PR-AUC Bar Chart
    model_names = [results[k]['name'] for k in pr_data.keys()]
    pr_auc_scores = [pr_data[k]['auc'] for k in pr_data.keys()]
    
    bars = ax2.bar(model_names, pr_auc_scores, color=colors[:len(model_names)], alpha=0.7)
    ax2.set_ylabel('PR-AUC Score')
    ax2.set_title('PR-AUC Comparison')
    ax2.set_ylim(0, 1)
    
    # Add value labels
    for bar, score in zip(bars, pr_auc_scores):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{score:.3f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('../models/trained_models/pr_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    return pr_data

pr_results = compute_pr_metrics(model_results)

## 4. Top-K Anomalies Analysis

In [ ]:
def analyze_top_k_anomalies(results, k_values=[50, 100, 200, 500]):
    """Analyze precision at top-k anomalies"""
    
    topk_data = {}
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
    
    for i, (model_name, data) in enumerate(results.items()):
        scores = data['scores']
        labels = data['labels']
        
        # Convert to anomaly scores and sort
        if model_name == 'svdd':
            anomaly_scores = -scores
        else:
            anomaly_scores = scores
        
        # Sort by anomaly score (descending)
        sorted_indices = np.argsort(anomaly_scores)[::-1]
        sorted_labels = labels[sorted_indices]
        
        # Calculate precision at top-k
        precisions_at_k = []
        for k in k_values:
            if k <= len(sorted_labels):
                precision_k = np.mean(sorted_labels[:k])
                precisions_at_k.append(precision_k)
            else:
                precisions_at_k.append(np.nan)
        
        topk_data[model_name] = {
            'k_values': k_values,
            'precisions': precisions_at_k
        }
        
        # Plot precision@k curve
        ax1.plot(k_values, precisions_at_k, 'o-', color=colors[i], 
                label=data['name'], linewidth=2, markersize=6)
    
    ax1.set_xlabel('Top-K Anomalies')
    ax1.set_ylabel('Precision@K')
    ax1.set_title('Precision at Top-K Anomalies')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(0, 1)
    
    # Bar chart for precision@100
    model_names = [results[k]['name'] for k in topk_data.keys()]
    p100_scores = [topk_data[k]['precisions'][1] for k in topk_data.keys()]  # k=100 is index 1
    
    bars = ax2.bar(model_names, p100_scores, color=colors[:len(model_names)], alpha=0.7)
    ax2.set_ylabel('Precision@100')
    ax2.set_title('Precision at Top-100 Anomalies')
    ax2.set_ylim(0, 1)
    
    # Add value labels
    for bar, score in zip(bars, p100_scores):
        if not np.isnan(score):
            ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                    f'{score:.3f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('../models/trained_models/topk_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    return topk_data

topk_results = analyze_top_k_anomalies(model_results)

## 5. False Positive Analysis

In [ ]:
def analyze_false_positives(results):
    """Analyze false positive rates and patterns"""
    
    fp_data = {}
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
    
    model_names = []
    fp_rates = []
    fn_rates = []
    
    for i, (model_name, data) in enumerate(results.items()):
        labels = data['labels']
        predictions = labels  # Using current predictions
        
        # Confusion matrix
        cm = confusion_matrix(labels, predictions)
        tn, fp, fn, tp = cm.ravel() if cm.size == 4 else (0, 0, 0, len(labels))
        
        # Calculate rates
        fp_rate = fp / (fp + tn) if (fp + tn) > 0 else 0
        fn_rate = fn / (fn + tp) if (fn + tp) > 0 else 0
        
        fp_data[model_name] = {
            'fp_rate': fp_rate,
            'fn_rate': fn_rate,
            'fp_count': fp,
            'fn_count': fn,
            'confusion_matrix': cm
        }
        
        model_names.append(data['name'])
        fp_rates.append(fp_rate)
        fn_rates.append(fn_rate)
    
    # False Positive Rate comparison
    bars1 = ax1.bar(model_names, fp_rates, color=colors[:len(model_names)], alpha=0.7)
    ax1.set_ylabel('False Positive Rate')
    ax1.set_title('False Positive Rate Comparison')
    ax1.set_ylim(0, max(fp_rates) * 1.1 if fp_rates else 1)
    
    for bar, rate in zip(bars1, fp_rates):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(fp_rates)*0.01,
                f'{rate:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # False Negative Rate comparison
    bars2 = ax2.bar(model_names, fn_rates, color=colors[:len(model_names)], alpha=0.7)
    ax2.set_ylabel('False Negative Rate')
    ax2.set_title('False Negative Rate Comparison')
    ax2.set_ylim(0, max(fn_rates) * 1.1 if fn_rates else 1)
    
    for bar, rate in zip(bars2, fn_rates):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(fn_rates)*0.01,
                f'{rate:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # Error rates heatmap
    error_matrix = np.array([fp_rates, fn_rates]).T
    im = ax3.imshow(error_matrix, cmap='Reds', aspect='auto')
    ax3.set_xticks([0, 1])
    ax3.set_xticklabels(['False Positive', 'False Negative'])
    ax3.set_yticks(range(len(model_names)))
    ax3.set_yticklabels(model_names)
    ax3.set_title('Error Rates Heatmap')
    
    # Add text annotations
    for i in range(len(model_names)):
        for j in range(2):
            ax3.text(j, i, f'{error_matrix[i, j]:.3f}', 
                    ha='center', va='center', color='white', fontweight='bold')
    
    plt.colorbar(im, ax=ax3)
    
    # Combined error rate (FP + FN)
    combined_errors = [fp + fn for fp, fn in zip(fp_rates, fn_rates)]
    bars4 = ax4.bar(model_names, combined_errors, color=colors[:len(model_names)], alpha=0.7)
    ax4.set_ylabel('Combined Error Rate (FP + FN)')
    ax4.set_title('Total Error Rate Comparison')
    
    for bar, rate in zip(bars4, combined_errors):
        ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(combined_errors)*0.01,
                f'{rate:.3f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('../models/trained_models/false_positive_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    return fp_data

fp_results = analyze_false_positives(model_results)

## 6. Comprehensive Results Summary

In [ ]:
def create_comprehensive_summary(roc_results, pr_results, topk_results, fp_results, model_results):
    """Create comprehensive comparison table and final recommendation"""
    
    # Create summary DataFrame
    summary_data = []
    
    for model_name, data in model_results.items():
        model_display_name = data['name']
        
        # Get metrics
        roc_auc = roc_results[model_name]['auc']
        pr_auc = pr_results[model_name]['auc']
        precision_100 = topk_results[model_name]['precisions'][1]  # Top-100
        fp_rate = fp_results[model_name]['fp_rate']
        fn_rate = fp_results[model_name]['fn_rate']
        
        # Anomaly detection rate
        anomaly_rate = np.mean(data['labels'])
        
        summary_data.append({
            'Model': model_display_name,
            'ROC-AUC': roc_auc,
            'PR-AUC': pr_auc,
            'Precision@100': precision_100,
            'False Positive Rate': fp_rate,
            'False Negative Rate': fn_rate,
            'Anomaly Rate': anomaly_rate
        })
    
    summary_df = pd.DataFrame(summary_data)
    
    # Display summary table
    print("\n" + "="*80)
    print("🏆 COMPREHENSIVE MODEL COMPARISON SUMMARY")
    print("="*80)
    print(summary_df.to_string(index=False, float_format='%.3f'))
    print("="*80)
    
    # Find best models for each metric
    best_roc = summary_df.loc[summary_df['ROC-AUC'].idxmax(), 'Model']
    best_pr = summary_df.loc[summary_df['PR-AUC'].idxmax(), 'Model']
    best_p100 = summary_df.loc[summary_df['Precision@100'].idxmax(), 'Model']
    lowest_fp = summary_df.loc[summary_df['False Positive Rate'].idxmin(), 'Model']
    
    print(f"\n🎯 BEST PERFORMERS:")
    print(f"   • Best ROC-AUC: {best_roc} ({summary_df[summary_df['Model']==best_roc]['ROC-AUC'].iloc[0]:.3f})")
    print(f"   • Best PR-AUC: {best_pr} ({summary_df[summary_df['Model']==best_pr]['PR-AUC'].iloc[0]:.3f})")
    print(f"   • Best Precision@100: {best_p100} ({summary_df[summary_df['Model']==best_p100]['Precision@100'].iloc[0]:.3f})")
    print(f"   • Lowest False Positives: {lowest_fp} ({summary_df[summary_df['Model']==lowest_fp]['False Positive Rate'].iloc[0]:.3f})")
    
    # Overall recommendation based on weighted score
    summary_df['Overall_Score'] = (
        0.3 * summary_df['ROC-AUC'] + 
        0.3 * summary_df['PR-AUC'] + 
        0.2 * summary_df['Precision@100'] + 
        0.2 * (1 - summary_df['False Positive Rate'])  # Lower FP is better
    )
    
    best_overall = summary_df.loc[summary_df['Overall_Score'].idxmax(), 'Model']
    best_score = summary_df['Overall_Score'].max()
    
    print(f"\n🏅 FINAL RECOMMENDATION: {best_overall}")
    print(f"   Overall Score: {best_score:.3f}/1.000")
    print(f"   (Weighted: 30% ROC-AUC + 30% PR-AUC + 20% P@100 + 20% Low-FP)")
    
    # Save summary
    summary_df.to_csv('../models/comprehensive_model_comparison.csv', index=False)
    print(f"\n💾 Saved: comprehensive_model_comparison.csv")
    
    return summary_df, best_overall

final_summary, recommended_model = create_comprehensive_summary(
    roc_results, pr_results, topk_results, fp_results, model_results
)

## 7. Final Visualization Dashboard

In [ ]:
def create_final_dashboard(final_summary):
    """Create a comprehensive dashboard visualization"""
    
    fig = plt.figure(figsize=(20, 12))
    gs = fig.add_gridspec(3, 4, hspace=0.3, wspace=0.3)
    
    # 1. ROC-AUC comparison
    ax1 = fig.add_subplot(gs[0, 0])
    bars1 = ax1.bar(final_summary['Model'], final_summary['ROC-AUC'], 
                   color=['#1f77b4', '#ff7f0e', '#2ca02c'][:len(final_summary)], alpha=0.8)
    ax1.set_title('ROC-AUC Scores', fontweight='bold')
    ax1.set_ylim(0, 1)
    for bar, score in zip(bars1, final_summary['ROC-AUC']):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                f'{score:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # 2. PR-AUC comparison
    ax2 = fig.add_subplot(gs[0, 1])
    bars2 = ax2.bar(final_summary['Model'], final_summary['PR-AUC'], 
                   color=['#1f77b4', '#ff7f0e', '#2ca02c'][:len(final_summary)], alpha=0.8)
    ax2.set_title('PR-AUC Scores', fontweight='bold')
    ax2.set_ylim(0, 1)
    for bar, score in zip(bars2, final_summary['PR-AUC']):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                f'{score:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # 3. Precision@100
    ax3 = fig.add_subplot(gs[0, 2])
    bars3 = ax3.bar(final_summary['Model'], final_summary['Precision@100'], 
                   color=['#1f77b4', '#ff7f0e', '#2ca02c'][:len(final_summary)], alpha=0.8)
    ax3.set_title('Precision@100', fontweight='bold')
    ax3.set_ylim(0, 1)
    for bar, score in zip(bars3, final_summary['Precision@100']):
        if not np.isnan(score):
            ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                    f'{score:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # 4. False Positive Rate
    ax4 = fig.add_subplot(gs[0, 3])
    bars4 = ax4.bar(final_summary['Model'], final_summary['False Positive Rate'], 
                   color=['#d62728', '#ff7f0e', '#2ca02c'][:len(final_summary)], alpha=0.8)
    ax4.set_title('False Positive Rate', fontweight='bold')
    for bar, score in zip(bars4, final_summary['False Positive Rate']):
        ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(final_summary['False Positive Rate'])*0.05,
                f'{score:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # 5. Radar chart for overall comparison
    ax5 = fig.add_subplot(gs[1, :2], projection='polar')
    
    metrics = ['ROC-AUC', 'PR-AUC', 'Precision@100', 'Low FP Rate']
    angles = np.linspace(0, 2 * np.pi, len(metrics), endpoint=False).tolist()
    angles += angles[:1]  # Complete the circle
    
    colors_radar = ['#1f77b4', '#ff7f0e', '#2ca02c']
    
    for i, (_, row) in enumerate(final_summary.iterrows()):
        values = [
            row['ROC-AUC'],
            row['PR-AUC'], 
            row['Precision@100'] if not np.isnan(row['Precision@100']) else 0,
            1 - row['False Positive Rate']  # Convert to "Low FP Rate"
        ]
        values += values[:1]
        
        ax5.plot(angles, values, 'o-', linewidth=2, label=row['Model'], 
                color=colors_radar[i], markersize=6)
        ax5.fill(angles, values, alpha=0.1, color=colors_radar[i])
    
    ax5.set_xticks(angles[:-1])
    ax5.set_xticklabels(metrics)
    ax5.set_ylim(0, 1)
    ax5.set_title('Overall Performance Radar', fontweight='bold', pad=20)
    ax5.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
    
    # 6. Summary table
    ax6 = fig.add_subplot(gs[1, 2:])
    ax6.axis('tight')
    ax6.axis('off')
    
    table_data = final_summary[['Model', 'ROC-AUC', 'PR-AUC', 'Precision@100', 
                               'False Positive Rate', 'Anomaly Rate']].round(3)
    
    table = ax6.table(cellText=table_data.values, colLabels=table_data.columns,
                     cellLoc='center', loc='center', bbox=[0, 0, 1, 1])
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 2)
    
    # Style the table
    for i in range(len(table_data.columns)):
        table[(0, i)].set_facecolor('#4CAF50')
        table[(0, i)].set_text_props(weight='bold', color='white')
    
    ax6.set_title('Performance Summary Table', fontweight='bold', pad=20)
    
    # 7. Recommendation box
    ax7 = fig.add_subplot(gs[2, :])
    ax7.axis('off')
    
    recommendation_text = f"""
🏆 FINAL RECOMMENDATION: {recommended_model}

Based on comprehensive evaluation across multiple metrics:
• ROC-AUC: Measures overall discrimination ability
• PR-AUC: Focuses on precision-recall trade-off (important for imbalanced data)
• Precision@100: Practical metric for investigating top anomalies
• False Positive Rate: Critical for operational deployment

The {recommended_model} demonstrates the best balance across all evaluation criteria.
    """
    
    ax7.text(0.5, 0.5, recommendation_text, transform=ax7.transAxes, 
            fontsize=12, ha='center', va='center',
            bbox=dict(boxstyle='round,pad=1', facecolor='lightblue', alpha=0.8))
    
    plt.suptitle('DAY 14: COMPREHENSIVE MODEL COMPARISON DASHBOARD', 
                fontsize=16, fontweight='bold', y=0.98)
    
    plt.savefig('../models/trained_models/final_comparison_dashboard.png', 
               dpi=150, bbox_inches='tight')
    plt.show()

create_final_dashboard(final_summary)

## 8. Day 14 Completion Summary

In [ ]:
print("\n" + "="*60)
print("✅ DAY 14 COMPLETED SUCCESSFULLY!")
print("="*60)

deliverables = [
    "📊 ROC-AUC comparison across all models",
    "📈 PR-AUC analysis for imbalanced data", 
    "🎯 Top-K anomalies precision evaluation",
    "❌ False positive rate analysis",
    "📋 Comprehensive comparison table",
    "🏆 Final model recommendation",
    "📱 Interactive comparison dashboard",
    "💾 All results saved to /models/"
]

print("\n🎯 DELIVERABLES COMPLETED:")
for deliverable in deliverables:
    print(f"   {deliverable}")

print(f"\n🏅 RECOMMENDED MODEL: {recommended_model}")
print("\n📁 FILES GENERATED:")
generated_files = [
    "roc_comparison.png",
    "pr_comparison.png", 
    "topk_comparison.png",
    "false_positive_analysis.png",
    "final_comparison_dashboard.png",
    "comprehensive_model_comparison.csv"
]

for file in generated_files:
    print(f"   ✅ {file}")

print("\n🚀 READY FOR DAY 15: Streamlit Dashboard Development!")
print("="*60)